In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
import joblib

# 1. Load and clean data
df = pd.read_csv('/workspaces/projects/CustomerChurn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Convert empty strings to NaN and clean numerical columns
for col in ['tenure', 'MonthlyCharges', 'TotalCharges']:
    df[col] = pd.to_numeric(df[col].replace(' ', np.nan), errors='coerce')

# Convert target variable
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# 2. Split data
X = df.drop(['Churn', 'customerID'], axis=1)  # Drop ID column
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Define feature types
numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
categorical_features = [
    'gender', 'Partner', 'Dependents', 'PhoneService', 
    'MultipleLines', 'InternetService', 'OnlineSecurity', 
    'OnlineBackup', 'DeviceProtection', 'TechSupport',
    'StreamingTV', 'StreamingMovies', 'Contract', 
    'PaperlessBilling', 'PaymentMethod'
]
binary_features = ['SeniorCitizen']

# 4. Create preprocessing pipelines
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# 5. Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('binary', binary_transformer, binary_features)
    ]
)

# 6. Create and train model pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        class_weight='balanced',
        random_state=42,
        n_estimators=150,
        max_depth=10
    ))
])

In [27]:
# 7. Train the model
pipeline.fit(X_train, y_train)

# 8. Evaluate
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

# 9. Save the pipeline
joblib.dump(pipeline, 'churn_pipeline_v2.pkl')

              precision    recall  f1-score   support

           0       0.89      0.81      0.85      1036
           1       0.58      0.73      0.65       373

    accuracy                           0.79      1409
   macro avg       0.74      0.77      0.75      1409
weighted avg       0.81      0.79      0.80      1409



['churn_pipeline_v2.pkl']